In [4]:
import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-1106")

alpha_vantage_api_key = os.environ.get("0Q58IAZITVOK3UIW")

class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        StockMarketSymbolSearchToolArgsSchema
    ] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


class CompanyOverviewArgsSchema(BaseModel):
    symbol: str = Field(
        description="Stock symbol of the company.Example: AAPL,TSLA",
    )


class CompanyOverviewTool(BaseTool):
    name = "CompanyOverview"
    description = """
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


class CompanyIncomeStatementTool(BaseTool):
    name = "CompanyIncomeStatement"
    description = """
    Use this to get the income statement of a company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()["annualReports"]


class CompanyStockPerformanceTool(BaseTool):
    name = "CompanyStockPerformance"
    description = """
    Use this to get the weekly performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        # CompanyIncomeStatementTool(),
        # CompanyStockPerformanceTool(),
    ],
)

prompt = "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Cloudflare stock'}`




/Users/jun/Agent/env/lib/python3.11/site-packages/langchain/utilities/duckduckgo_search.py:43: UserWarning: DDGS running in an async loop. This may cause errors. Use AsyncDDGS instead.
  with DDGS() as ddgs:


Cloudflare (NET) concluded the recent trading session at $82.96, signifying a -1.59% move from its prior day's close. 2d ago Apple quarterly profit, revenue top Wall Street targets but China lags... Stock analysis for Cloudflare Inc (NET:New York) including stock price, stock chart, company news, key statistics, fundamentals and company profile. 2,286,155 Today's Open 81.24 Previous Close 81.86 Intraday High 82.50 Intraday Low 79.80 52 Week High 87.17 52 Week Low 39.90 Pricing delayed by 20 minutes. Last Updated Jan 26, 2024 4:00 PM ET Stock Chart Zoom 1m 3m 6m YTD 1y All From Oct 26, 2023 To Jan 26, 2024 NYSE:NET News Releases 30. Oct 6. Nov 13. Nov 20. Nov 27. Nov 4. Dec 11. Dec 18. Cloudflare is a company that provides cloud-based services for web security, performance and availability. The stock price, news, analysis and historical data of Cloudflare (NET) are available on Markets Insider. See the latest ratings, forecasts, dividends and insider activity of Cloudflare stock. Cloudf

{'input': "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment.",
 'output': "Cloudflare Inc (NET) is a technology company that operates a cloud platform offering a range of network services to companies worldwide. Here are some key financial metrics and stock performance details for Cloudflare:\n\n- Market Capitalization: $26,542,778,000\n- EBITDA: -$81,727,000\n- P/E Ratio: Not available\n- PEG Ratio: 1.945\n- Book Value: $2.083\n- Dividend Per Share: $0\n- Dividend Yield: 0%\n- EPS (Earnings Per Share): -$0.61\n- Revenue Per Share (TTM): $3.645\n- Profit Margin: -16.7%\n- Operating Margin (TTM): -11.7%\n- Return on Assets (TTM): -4.66%\n- Return on Equity (TTM): -31.2%\n- Revenue (TTM): $1,208,972,000\n- Gross Profit (TTM): $742,631,000\n- Diluted EPS (TTM): -$0.61\n- Quarterly Earnings Growth (YOY): 0%\n- Quarterly Revenue Growth (YOY): 32.2%\n- Analyst Target Pr